# Parsing of the bokeh layout playing

In [ ]:
import pyparsing 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import *
from bokeh.models import HoverTool
output_notebook()
import random
from bokeh.palettes import Spectral6 as palette
import itertools
colors = itertools.cycle(palette)

In [ ]:
npoints=50
nfigures=10
x = list(range(npoints))
yList=[]
figList=[]
tooltips=[("x","(@x)"),  ("y","@y")]
tools = 'pan,box_zoom, wheel_zoom,box_select,lasso_select,reset'
tools = [HoverTool(tooltips=tooltips), tools]
for i in range(nfigures):
    y=random.sample(xrange(100), npoints)
    if (i>0): 
        for idx, val in enumerate(y):
            y[idx]+=yList[i-1][idx]
    y.sort()
    yList.append(y)
    f = figure(width=250, plot_height=250,tools=tools)
    f.circle(x, y, size=10, color=next(colors), alpha=0.5)
    figList.append(f)
f=figList    

In [ ]:
f[2].plot_width=2*f[0].plot_width
pp=gridplot([[f[0],f[1]],[f[2]],[f[3]]],plot_height=150)
show(pp)

In [ ]:
def processRow(row, figList,layoutList):
    print("Raw",row)
    array=[]
    layoutList.append(array)
    option=processLayoutOption(row)
    print("Option",option)
    for idx,y in enumerate(row):
        if not y.isdigit(): continue 
        fig=figList[int(y)]
        array.append(fig)
        if ('commonY' in option):
            if type(option["commonY"])==str: 
                fig.y_range=array[0].y_range
            else:
                try: fig.y_range=figList[int(option["commonY"])].y_range
                except ValueError: continue
        if ('commonX' in option):
            if type(option["commonX"])==str: 
                fig.x_range=array[0].x_range
            else:
                try: fig.x_range=figList[int(option["commonX"])].x_range
                except ValueError,e : 
                    logger.error('Failed: to processs option '+ option["commonX"])

            
        if (idx>0) & ('y_visible' in option): fig.yaxis.visible=bool(option["y_visible"])
        if ('x_visible' in option):     fig.xaxis.visible=bool(option["x_visible"])
            
           
def processLayoutOption(layout):   # https://stackoverflow.com/questions/9305387/string-of-kwargs-to-kwargs
    options={}
    for x in layout: 
        if not(type(x) == str): continue
        if ("=" in str(x)):                  ## one of the way to see if it's list
            try:
                k, v=x.split("=")
            except ValueError: continue   
            options[k]=v
            if v.isdigit():
                options[k]=int(v)
            else:
                try:
                    options[k]=float(v)
                except ValueError:
                    options[k]=v 
    return options

def processLayout(layout, figList):
    thecontent = pyparsing.Word(pyparsing.alphanums+".+-=_")  | pyparsing.Suppress(',')
    parens     = pyparsing.nestedExpr( '(', ')', content=thecontent)
    res = parens.parseString(layout)[0]
    layoutList=[]
    optionList=[]
    #print(res.asList())
    options=processLayoutOption(res)
    print(options)
    for x in res:
        if type(x) != str:  
           processRow(x,figList,layoutList)
    return res.asList(), layoutList, options

In [ ]:
layout="((0,3,commonY=1,commonX=1.4,x_visible=1,y_visible=0), (1,plot_height=0.4,commonY=1,commonX=1), plot_height=300, plot_width=600)"
x,layoutList,options=processLayout(layout,figList)
show(gridplot(layoutList,**options))